<a href="https://colab.research.google.com/github/priyaChaturvedi11/Face_Recognition_Project/blob/master/snn_oneshort_using_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
# os package is used to read files and directory structure
import os
# NumPy is used to convert python list to numpy array and to perform required matrix operations
import numpy as np
# matplotlib.pyplot to plot the graph and display images in the training and validation data
import matplotlib.pyplot as plt

# to read images
import cv2

# using pickle module to save image tensors
import pickle

# for Image.open()
from PIL import Image


In [ ]:
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng

Using TensorFlow backend.


In [ ]:
PATH = "/content/drive/My Drive/Colab Notebooks/frs_dataset_w_train_test_split"

In [ ]:
train_dir = os.path.join(PATH, 'att_training')
validation_dir = os.path.join(PATH, 'att_testing')
save_path = '/content/drive/My Drive/Colab Notebooks/data'

In [ ]:
train_dir

'/content/drive/My Drive/Colab Notebooks/frs_dataset_w_train_test_split/att_training'

In [ ]:
validation_dir

'/content/drive/My Drive/Colab Notebooks/frs_dataset_w_train_test_split/att_testing'

In [ ]:
save_path

'/content/drive/My Drive/Colab Notebooks/data'

Understanding the data

In [ ]:
num_total_classes_for_training = len(os.listdir(train_dir))
num_total_classes_for_testing = len(os.listdir(validation_dir))

In [ ]:
num_total_classes_for_testing

3

In [ ]:
num_total_classes_for_training
# earlier it was 37, we deleted S3

36

In [ ]:
total_images_for_training = 0
total_images_in_folder = 0
for folder in os.listdir(train_dir):
    # print("Loading folder:" + folder)
    folder_path = os.path.join(train_dir,folder)
    total_images_in_folder = len(os.listdir(folder_path))
    # print("Total images in this folder", total_images_in_folder)
    total_images_for_training += total_images_in_folder
print("Total images for training: ", total_images_for_training)
# 14 * 36 = 504

Total images for training:  504


To see what loadimgs() function is doing
And what do X and y contain


In [ ]:
    path = validation_dir
    X = []
    y = []
    folder_index_dictionary = {}
    # to indicate folder index
    n = 0
    curr_y = n

    # do this for all the folders in testing set 
    for folder in os.listdir(path):
        print("loading folder" + folder, curr_y)
        # folder_index_dictionary contains keys such as S1, S2, S3
        # for each key there is a list of length 2
        # first element of that list contains index of the folder
        # second index of list contains None and 
        # will later contain a value equal to curr_y-1 because after every iteration we are increasing curr_y
        folder_index_dictionary[folder] = [curr_y, None]
        folder_path = os.path.join(path, folder)
        all_image_instances = []
        

        # do this for all the images in a folder
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            X.append(
                [np.asarray(
                    Image.open(image_path).crop((10, 0, 92, 102)).resize((64,64), Image.ANTIALIAS)
                ), curr_y]
            )
            y.append(curr_y)
        curr_y += 1
        # what is this doing??
        folder_index_dictionary[folder][1] = curr_y - 1
        print("folder_index_dictionary", folder_index_dictionary)
        # print(X[0][0][0])


loading folderS7 0
folder_index_dictionary {'S7': [0, 0]}
loading folderS5 1
folder_index_dictionary {'S7': [0, 0], 'S5': [1, 1]}
loading folderS6 2
folder_index_dictionary {'S7': [0, 0], 'S5': [1, 1], 'S6': [2, 2]}


In [ ]:
image = cv2.imread(image_path)
            all_image_instances.append(image)
            # for each image in testing set, y contains index of folder to which they belong
            y.append(curr_y)
        print(y)   
        try:
            X.append(np.stack(all_image_instances, axis = 0))
            print(len(X))
        # edge case - last one
        except ValueError as e:
            print(e)
            print("error - all image instances", all_image_instances)

In [ ]:
y

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2]

In [ ]:
    y = np.vstack(y)
    y

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2]])

In [ ]:
X

[[array([[51, 54, 55, ..., 58, 60, 59],
         [55, 56, 55, ..., 57, 59, 61],
         [57, 54, 56, ..., 59, 60, 60],
         ...,
         [51, 49, 52, ..., 64, 63, 63],
         [48, 47, 51, ..., 65, 64, 61],
         [47, 51, 50, ..., 64, 64, 61]], dtype=uint8), 0],
 [array([[54, 55, 55, ..., 60, 58, 57],
         [57, 56, 56, ..., 61, 61, 57],
         [56, 54, 57, ..., 61, 61, 60],
         ...,
         [53, 48, 51, ..., 63, 64, 65],
         [52, 50, 50, ..., 63, 68, 64],
         [49, 49, 51, ..., 64, 66, 64]], dtype=uint8), 0],
 [array([[52, 52, 52, ..., 57, 57, 52],
         [53, 54, 52, ..., 56, 59, 57],
         [53, 52, 53, ..., 56, 55, 58],
         ...,
         [48, 48, 50, ..., 59, 60, 60],
         [49, 50, 51, ..., 58, 61, 62],
         [50, 52, 50, ..., 58, 59, 62]], dtype=uint8), 0],
 [array([[73, 70, 74, ..., 73, 71, 75],
         [74, 79, 85, ..., 75, 74, 76],
         [81, 82, 86, ..., 75, 74, 73],
         ...,
         [55, 54, 57, ..., 75, 78, 75],
       

In [ ]:
len(X)

43

In [ ]:
X = np.stack(X)       
X

array([[array([[51, 54, 55, ..., 58, 60, 59],
       [55, 56, 55, ..., 57, 59, 61],
       [57, 54, 56, ..., 59, 60, 60],
       ...,
       [51, 49, 52, ..., 64, 63, 63],
       [48, 47, 51, ..., 65, 64, 61],
       [47, 51, 50, ..., 64, 64, 61]], dtype=uint8),
        0],
       [array([[54, 55, 55, ..., 60, 58, 57],
       [57, 56, 56, ..., 61, 61, 57],
       [56, 54, 57, ..., 61, 61, 60],
       ...,
       [53, 48, 51, ..., 63, 64, 65],
       [52, 50, 50, ..., 63, 68, 64],
       [49, 49, 51, ..., 64, 66, 64]], dtype=uint8),
        0],
       [array([[52, 52, 52, ..., 57, 57, 52],
       [53, 54, 52, ..., 56, 59, 57],
       [53, 52, 53, ..., 56, 55, 58],
       ...,
       [48, 48, 50, ..., 59, 60, 60],
       [49, 50, 51, ..., 58, 61, 62],
       [50, 52, 50, ..., 58, 59, 62]], dtype=uint8),
        0],
       [array([[73, 70, 74, ..., 73, 71, 75],
       [74, 79, 85, ..., 75, 74, 76],
       [81, 82, 86, ..., 75, 74, 73],
       ...,
       [55, 54, 57, ..., 75, 78, 75],
   

In [ ]:
print(len(all_image_instances))

14


In [ ]:
all_image_instances

In [ ]:
count = 1
for all_images in all_image_instances:
    print(all_images.shape, count, all_images.dtype)
    count += 1

(112, 92, 3) 1 uint8
(112, 92, 3) 2 uint8
(112, 92, 3) 3 uint8
(112, 92, 3) 4 uint8
(112, 92, 3) 5 uint8
(112, 92, 3) 6 uint8
(112, 92, 3) 7 uint8
(112, 92, 3) 8 uint8
(112, 92, 3) 9 uint8
(112, 92, 3) 10 uint8
(112, 92, 3) 11 uint8
(112, 92, 3) 12 uint8
(112, 92, 3) 13 uint8
(112, 92, 3) 14 uint8


In [ ]:
all_image_instances[0].shape
# each image is currentlt an array of shape 112 * 92

(112, 92, 3)

In [ ]:
count = 1
for all_folders in folder_index_dictionary:
    print(all_folders, count)
    count += 1

S7 1
S5 2
S6 3


In [ ]:
print(len(folder_index_dictionary))

3


y has indices, 0 if image belongs to folder 1, and so on

In [ ]:
print(len(y))
# 3 * 14 = 42
# why is it showing 43??

43


To see what does X contain

In [ ]:
len(X)
# three batches of 14 images for each folder = 3 * 14 = 42

43

Function to load images and Data Engineering

In [ ]:
def loadimgs (path, n = 0):
    X = []
    y = []
    folder_index_dictionary = {}
    # to indicate folder index
    curr_y = n

    # do this for all the folders in training set 
    for folder in os.listdir(path):
        print("loading folder" + folder, curr_y)
        # folder_index_dictionary contains keys such as S1, S2, S3
        # for each key there is a list of length 2
        # first element of that list contains index of the folder
        # second index of list contains None and 
        # will later contain a value equal to curr_y-1
        folder_index_dictionary[folder] = [curr_y, None]
        folder_path = os.path.join(path, folder)

        # do this for all the images in a folder
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            X.append(
                [np.asarray(
                    Image.open(image_path).crop((10,0,92,102)).resize((64,64), Image.ANTIALIAS)
                ), curr_y]
            )
            y.append(curr_y)
        curr_y += 1
        # what is this doing??
        # decrementing so that previous value could be stored
        folder_index_dictionary[folder][1] = curr_y - 1
    y = np.vstack(y)
    X = np.stack(X)
    return X, y, folder_index_dictionary 
          

Loading the train images into tensors

In [ ]:
X, y, c = loadimgs(train_dir)

In [ ]:
print(X.shape)
# X contains 36 folders and each folder has 14 images
# so 36 * 14 = 504 rows
# 2 columns, one for image tensors, one for labels

(504, 2)


In [ ]:
print(X[0].shape)

(2,)


Saving the train tensors on disk

In [ ]:
with open(os.path.join(save_path,"train.pickle"), "wb") as f:
    pickle.dump((X,c),f)

Loading the validation images into tensor

In [ ]:
Xval,yval,cval=loadimgs(validation_dir)

loading folderS7 0
loading folderS5 1
loading folderS6 2


Saving the validation tensors on disk

In [ ]:
with open(os.path.join(save_path,"val.pickle"), "wb") as f:
    pickle.dump((Xval,cval),f)

In [ ]:
def initialize_weights(shape, name=None):
    
        # The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        # suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01

    # as mentioned in Harshall Lamba's blog
    # https://github.com/hlamba28/One-Shot-Learning-with-Siamese-Networks/blob/master/Siamese%20on%20Omniglot%20Dataset.ipynb
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [ ]:
def initialize_bias(shape, name=None):
    
       # The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
       # suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

Building Siamese Model

In [ ]:
def get_siamese_model(input_shape):
    
    # Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape = input_shape,
                   kernel_initializer = initialize_weights, kernel_regularizer = l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation = 'relu',
                     kernel_initializer = initialize_weights,
                     bias_initializer = initialize_bias, kernel_regularizer = l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation = 'relu', kernel_initializer = initialize_weights,
                     bias_initializer = initialize_bias, kernel_regularizer = l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation = 'relu', kernel_initializer = initialize_weights,
                     bias_initializer = initialize_bias, kernel_regularizer = l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation = 'sigmoid',
                   kernel_regularizer = l2(1e-3),
                   kernel_initializer = initialize_weights,bias_initializer = initialize_bias))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation = 'sigmoid', bias_initializer = initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs = [left_input,right_input], outputs = prediction)
    
    # return the model
    return siamese_net

In [ ]:
model = get_siamese_model((64, 64, 1))
model.summary()

TypeError: ignored

Making pairs